In [1]:
import random, gym
import numpy as np
from deap import algorithms, base, creator, tools

## Simülasyon ortamı

In [2]:
env = gym.make("CartPole-v1")
observation_space_dim = env.observation_space.shape[0]
action_space_dim = env.action_space.n

#### Tekrarlanabilirlik için seed ayarlanır

In [3]:
SEED_VALUE = 32
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
env.seed(SEED_VALUE)

[32]

#### Simülasyonu çalıştırmak için ortak fonksiyon

In [4]:
def run_env(act_fn, n_episode=1, render=False):
	# Episode döngüsü
	totalRewards = 0
	for episode in range(n_episode):
		state = env.reset()
		done = False

		# Timestep döngüsü
		while not done:
			if render: env.render()

			# act_fn adlı fonksiyon parametreyi çağırarak aksiyonu al
			action = act_fn(state)
			next_state, reward, done, info = env.step(action)

			totalRewards += reward
			state = next_state
	return totalRewards

## Yapay Sinir Ağı

#### Ağırlıkların parametre sayısı

In [5]:
nn_hidden_unit = 8
w1_ndim = (observation_space_dim*nn_hidden_unit)
w2_ndim = (nn_hidden_unit*action_space_dim)

#### Verilen bireyin genotipini, yapay sinir ağı parametreleri olarak kullanarak state vektörünü ağa feed-forward eder

In [6]:
def nn_forward(individual, state):
	# State'in 2 seviyeli array olduğundan emin ol: [?, observation_space_dim]
	if len(state.shape) == 1: state = np.array([state])
	assert len(state.shape) == 2

	# Yapay sinir ağı parametrelerini genotip olan 1D vektörden çıkar
	individual = np.array(individual)
	w1 = individual[:w1_ndim]
	b1 = individual[w1_ndim]
	w2 = individual[w1_ndim+1:-1]
	b2 = individual[-1]

	# Ağırlıkları matris çarpımı için yeniden şekillendir
	w1 = np.reshape(w1, (observation_space_dim, nn_hidden_unit))
	w2 = np.reshape(w2, (nn_hidden_unit, action_space_dim))

	# Sigmoid fonksiyonu
	sigmoid = lambda x: (1 / (1 + np.exp(-x)))

	# Feed-forward
	h1 = sigmoid(np.dot(state, w1) + b1)
	output = sigmoid(np.dot(h1, w2) + b2)
	return np.argmax(output)

## Genetik Algoritmalar

### Hiperparametreler

In [7]:
# Her bireyin gen uzunluğu (ağın parametre sayısı)
n_features = (w1_ndim + w2_ndim + 2)

# Simüle edilecek nesil sayısı
n_generation = 25

# Popülasyon boyutu (birey sayısı)
n_population = 32

# Seleksiyon turnuvasındaki birey sayısı
selectionTournamentSize = 3

# Çaprazlama ve mutasyon olasılıkları
crossingoverProbability = 0.50
mutationProbability = 0.10

In [8]:
# Uygunluk ve Birey sınıflarını oluştur
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

# Bireylerin genotiplerini ve popülasyonun basitçe birey listesi olduğunu tanımla
toolbox = base.Toolbox()
toolbox.register("attr_weights_n_biases", random.gauss, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_weights_n_biases, n_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#### Uygunluk fonksiyonunu tanımla

In [9]:
def calculateFitness(individual):
	individualEpisodeRewards = run_env(
		# Aksiyon fonksiyonumuz, bireyin genotipini kullanarak yapay sinir ağını çalıştırır
		act_fn=(lambda state: nn_forward(individual, state)),
		# Her uygunluk hesabında ortamı 3 episode çalıştır
		n_episode=3
	)
	# tuple tipinde çevirmeli
	return (individualEpisodeRewards, )

In [10]:
# Uygunluk fonksiyonunu kaydet
toolbox.register("evaluate", calculateFitness)

# Hangi çaprazlama, mutasyon ve seçilim yöntemlerinin kullanılacağını tanımla
toolbox.register("mate", tools.cxUniform, indpb=0.50)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=selectionTournamentSize)

#### Popülasyonu oluştur

In [11]:
population = toolbox.population(n=n_population)
stats = tools.Statistics(lambda ind: ind.fitness.values[0])
hallOfFame = tools.HallOfFame(1)

## Simülasyon başlasın!

In [12]:
finalPopulation, logs = algorithms.eaSimple(
	population=population,
	toolbox=toolbox,
	halloffame=hallOfFame,
	stats=stats,
	ngen=n_generation,
	cxpb=crossingoverProbability,
	mutpb=mutationProbability,
	verbose=True
)

gen	nevals
0  	32    
1  	14    
2  	13    
3  	16    
4  	23    
5  	23    
6  	14    
7  	13    
8  	16    
9  	21    
10 	23    
11 	17    
12 	20    
13 	19    
14 	20    
15 	21    
16 	17    
17 	12    
18 	20    
19 	14    
20 	14    
21 	19    
22 	14    
23 	18    
24 	23    
25 	17    


### En iyi bireyle simülasyonu çalıştır

In [13]:
best = hallOfFame[0]
print("[+] En iyi uygunluk değeri: {}".format(best.fitness.values[0]))
print("Genotip: ", best)

run_env(
	act_fn=(lambda state: nn_forward(best, state)),
	n_episode=5,
	render=True
)
env.close()

[+] En iyi uygunluk değeri: 1500.0
Genotip:  [1.3058483999025259, 0.5239581436378747, -0.9934185979558662, -1.2136314573096108, 0.9322521455993511, -1.7907323769385324, 0.6996398988979424, 1.3686562713792414, 1.0430222805678417, 0.08612360013032382, 0.3981610689973498, -0.7191259097412133, -0.2826876446536739, 1.9476190031354246, 1.150752330383713, -1.109570969846198, -0.7140813750130052, 0.7896447711648813, 1.9101949103686673, -0.4963878680155234, 0.11624376288143501, 1.4811352578821997, 0.5859399858745413, 0.3255695794696582, 0.2444314847660672, -0.3351945637773304, 1.6069888989404995, -0.7177252162701241, -1.405550906585943, 0.4465994552915829, -0.6001821614442142, 0.02806603354988558, -1.0223654917091007, -0.965107240937428, 1.818605177222585, 0.12146933171565409, 1.5022902213472842, -1.0249682094540045, 0.8896268085173817, -0.6621325075909323, -2.6942556188636533, 0.967839690869904, -2.5327403361921292, -1.6086439052590682, -0.5464178448073405, 2.0642782977705076, -0.4884786293128